## EDA

In [1]:
import os
from math import sqrt, pi
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

In [2]:
df = pd.read_parquet('grouped.parquet')

df.loc[:, 'sensors|pois'] = df['sensors'] / df['pois']
df.loc[:, 'covg|comm'] = df['coverage_r'] / df['communication_r']
df.loc[:, 'covg|area'] = (df['coverage_r']**2*pi) / (df['area_side']**2)
df.loc[:, 'covg*sensors|area'] = ((df['coverage_r']**2*pi)*df['sensors']) / (df['area_side']**2)

In [4]:
! pip install openpyxl

     |████████████████████████████████| 243 kB 1.3 MB/s eta 0:00:01


In [5]:
df.to_excel('data.xlsx', index=None)

In [ ]:
#view = df.groupby('sensors|pois').mean().sort_index()
#view = df.groupby('covg|comm').mean().sort_index()
#view = df.groupby('covg|area').mean().sort_index()
view = df.groupby('covg*sensors|area').mean().sort_index()

fig = go.Figure(data=[
    go.Scatter(x=view.index, y=view[item], name=item)
    for item in ['K=1', 'K=2', 'K=3', 'K=4', 'K=5']
] + [
    go.Scatter(x=view.index, y=view[item], name=item)
    for item in ['M=1', 'M=2', 'M=3']
])
fig.show()

## RE-SET THE PARAMETERS

In [ ]:
POIS         = [10,  20,   30,  40,   50]
RATE_SENSORS = [2.5, 5.0,  10.0]
RATE_SINKS   = [0.0, 0.05, 0.1]
RATE_AREA    = [8.0, 2.0, 1.0]  # HOW MUCH % of the total area we can COVER
COVERAGE_R   = 100  # BASE-CONSTANT
RATE_COMM_R  = [0.5, 1.0,  1.5, 2.0]

from math import sqrt, pi
combinations = []
covg_r = COVERAGE_R
for pois in POIS:
    for sensors in [int(pois*r) for r in RATE_SENSORS]:
        for sinks in [int(max(sensors*r, 1)) for r in RATE_SINKS]:
            for area in [int(sqrt((1/r)*sensors*pi*covg_r*covg_r)) for r in RATE_AREA]:
                for comm_r in [int(covg_r*r) for r in RATE_COMM_R]:
                    combinations.append((pois, sensors, sinks, area, covg_r, comm_r))
combinations = pd.DataFrame(
    combinations,
    columns=['num_pois', 'num_sensors', 'num_sinks', 'area_side', 'covg_radius', 'comm_radius']
)
print(len(combinations))
combinations = (combinations
    .drop_duplicates()
    .sort_values(list(combinations.columns))
    .reset_index(drop=True)
).copy()
len(combinations)

In [ ]:
# combinations.to_csv('instance_generator_configurations.csv', sep=',', index=None)

In [ ]:
combinations